In [1]:
# earth engine jupyter အား install ပြုလုပ်ခြင်း
try:
  import ee_jupyter
  print('ee_jupyter was already installed.')
except ModuleNotFoundError:
  print('ee_jupyter was not found. Installing now...')
  import os
  result = os.system('pip -q install earthengine-jupyter')

ee_jupyter was already installed.


# earth engine နဲ့ လိုအပ်သော libraries များအား import လုပ်ခြင်း

In [2]:
# earth engine account အား access လုပ်ခြင်း
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=46TxThgjC41AAT8w6P37_oN9m_l4KyQaqtxFZlGi-jI&tc=9jolREm66xZfkRBKUw87YC0SFiHZBG0cu7hHY1txcdA&cc=gxKxMZa1iyrhI6gNxFap3z65hB0XnUoJMsIW-6bF2pA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvsGRRX2N_52g3ODId_BPCIUs6Q086iRHqrXLhN1G7wh-uhG298AZwM

Successfully saved authorization token.


In [3]:
import altair as alt
from ee_jupyter.colab import set_colab_output_cell_height
from ee_jupyter.ipyleaflet import Map
from ee_jupyter.ipyleaflet import Inspector
from ee_jupyter.layout import MapWithInspector as MapPlus
import ipyleaflet
import ipywidgets as widgets
from IPython.display import HTML
import math
import numpy as np
import pandas as pd
from pprint import pprint

## DEM အားအသုံးပြုခြင်း

In [4]:
dem=ee.Image('CGIAR/SRTM90_V4')

In [5]:
set_colab_output_cell_height(300)
pprint(dem.getInfo())

<IPython.core.display.Javascript object>

{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [0.000833333333333,
                              0,
                              -180,
                              0,
                              -0.000833333333333,
                              60],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [432000, 144000],
            'id': 'elevation'}],
 'id': 'CGIAR/SRTM90_V4',
 'properties': {'date_range': [950227200000, 951177600000],
                'description': '<p>The Shuttle Radar Topography Mission (SRTM) '
                               'digital\n'
                               'elevation dataset was originally produced to '
                               'provide consistent,\n'
                               'high-quality elevation data at near global '
                               'scope. This version\n'
    

In [6]:
location=[24.18222,96.33056] #katha township location
map_init_params={
    'center':list(location), #lat,lon
    'zoom':10
}

In [7]:
map1=Map(**map_init_params)
map1

Map(center=[24.18222, 96.33056], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [8]:
# ပထမလုပ်ထားတဲ့ နေရာကို layer အသစ်ပေါင်းထည့်ခြင်း
map1.addLayer(dem,{'min':0,'max':1000},'elevation(greyscale)')

In [9]:
map1.addLayer(
    dem,
    {'min':0,'max':1000,'palette':['grey','yellow','red']},
    'elevation(custom palette)'
)

## computation using images

In [10]:
# apply an algorithm to an image
slope=ee.Terrain.slope(dem)

In [11]:
#  display
map2a=Map(**map_init_params)
map2a.addLayer(dem,{'min':0,'max':1000},'elevation[meters]')

In [12]:
# slope is calculated in degrees so 30 degrees was chosen as the max for this
# area. In general, is rare to go above 45 degrees.
map2a.addLayer(slope,{'min':0,'max':45},'slope[degrees')
map2a

Map(center=[24.18222, 96.33056], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

## Using Map Inspector

In [13]:
inspector2a=Inspector(map2a)
widgets.HBox([map2a,inspector2a])

In [14]:
# နောက်ထပ် layer ထည့်နည်း
map_panel_2=MapPlus(**map_init_params)
map_panel_2.map.addLayer(dem,{'min':0,'max':1000},'elevation[meters]')
map_panel_2.map.addLayer(slope,{'min':0,'max':30},'slope [degrees]')
map_panel_2

MapWithInspector(children=(HBox(children=(Map(center=[24.18222, 96.33056], controls=(ZoomControl(options=['pos…

## Aspect

The aspect is the orientation of the slope, measured clockwise in degrees from 0 to 360, where 0 is north-facing, 90 is east-facing, 180 is south-facing and 270 is west-facing.

In [15]:
aspect=ee.Terrain.aspect(dem)

In [16]:
# convert to radians, compute the sine of the aspect
sinImage=aspect.divide(180).multiply(np.pi).sin()

In [17]:
# Display maps side-by-side

In [18]:
map_params_mount_popa={
    'center':(20.921162982,95.250332332),
    'zoom':10
}
map2b=Map(**map_params_mount_popa)
map2b.addLayer(dem,{'min':0,'max':4000},'elevation [meters]')
map2c=Map(**map_params_mount_popa)
map2c.addLayer(
    sinImage,
    {'min':-1,'max':1,'palette':['yellow','white','red']},
    'sine of aspect'
)
# west...yellow, white.....north/south, east.......red
widgets.HBox([map2b,map2c])

In [19]:
# center အားပြန်ကြည့်ခြင်း
map2b.center

[20.921162982, 95.250332332]

### Image statistics

In [20]:
# use ipyleaflet to add the ability to draw a geometry on map
draw_control=ipyleaflet.DrawControl(
    rectangle={},
    polyline={},
    circlemarker={}
)
def handle_draw(target,action,geo_json):
  with output:
    output.clear_output()
    pprint(geo_json)
  draw_control.on_draw(handle_draw)

map2d=Map(**map_init_params)
output=widgets.Output(layout={'border':'1px solid black','width':'200'})
map2d.addLayer(dem,{'min':0,'max':1000},'elevation [meters]')
map2d.add_control(draw_control)
widgets.VBox([map2d,output])

In [26]:
geom_clientside=draw_control.last_draw['geometry']
geom_clientside

{'type': 'Polygon',
 'coordinates': [[[96.265869, 24.261768],
   [96.262207, 24.342704],
   [96.373901, 24.35605],
   [96.405945, 24.310169],
   [96.38855, 24.260933],
   [96.378479, 24.23923],
   [96.265869, 24.261768]]]}

##  Spatial reduction: calculate stats for custom geometry

In [27]:
# create an earth engine server-side geometry
geom=ee.Geometry(geom_clientside)

In [28]:
# compute mean elevation in the polygon
meandict=dem.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=geom,
    scale=90,
    bestEffort=True
)

# get the mean from the dictionary and print it
mean=meandict.get('elevation')
print('Mean elevation',mean.getInfo())

Mean elevation 295.54582207295994
